<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:#5642C5;
           font-size:200%;
           font-family:Arial;letter-spacing:0.5px">

<p width = 20%, style="padding: 10px;
              color:white;">
Aggregating And Combining DataFrames              
</p>
</div>

Data Science Cohort Live NYC May 2022
<p>Phase 1: Topic 5</p>
<br>
<br>

<div align = "right">
<img src="Images/flatiron-school-logo.png" align = "right" width="200"/>
</div>
    
    

## Objectives

- Use GroupBy objects to organize and aggregate data
- Create pivot tables from DataFrames
- Combine DataFrames by merging, joining, and concatinating

Categorical variable taking on a few discrete values.

Each of these values form a group. Want to:
- Calculate statistics on various quantities for each group (mean, etc.)
- Transform/scale certain columns differently for each group.


DataFrame.groupby() allows us to do this.

Take the Titanic dataset again:

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

titanic_df = pd.read_csv('Data/titanic.csv')
titanic_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Sex as  relevant categorical variable:
- survival rate
- distribution of ages
- fare

# groupby 

In [7]:
titanic_subset = titanic_df[['Sex', 'Survived', 'Age', 'Fare']]
titanic_subset.groupby('Sex')

groupby object has many useful methods for processing data by group.

#### Aggregation methods 

- Methods that compute statistics across the different groups.
- Common aggregation methods:
    - .min(): returns the minimum value for each column by group
    - .max(): returns the maximum value for each column by group
    - .mean(): returns the average value for each column by group
    - .median(): returns the median value for each column by group
    - .count(): returns the count of each column by group
    - .sum(): return sum of each column by group

Computing the mean of columns by group:
- Note: mean of Survived is the survival fraction.

In [8]:
titanic_subset.groupby('Sex').mean()

,Survived,Age,Fare
Sex,,,
female,0.742038,27.915709,44.479818
male,0.188908,30.726645,25.523893


In [9]:
titanic_subset.groupby('Sex')['Fare'].mean()

Sex
female    44.479818
male      25.523893
Name: Fare, dtype: float64

Any obvious distinctions between groups here?

#### .agg(func) method
Can write your own aggregations.
- Get square root of the sum of squares of desired columns.

In [10]:
titanic_subset.groupby('Sex').agg(lambda x: np.sqrt(np.sum(x**2)))

,Survived,Age,Fare
Sex,,,
female,15.264338,505.132532,1293.863603
male,10.440307,724.618934,1203.237998


#### .transform(func) method
- This is not an aggregation.
- Transforms entries in each column differently according to their group.




Example: standardize columns for each sex separately:

- Subtract entries of columns in each sex category by the column mean for that sex.
- Then divide by the standard deviation of fare for that sex.

In [11]:
titanic_subset.groupby('Sex').transform(lambda col: (col - col.mean())/col.std(ddof = 1) )

,Survived,Age,Fare
0,-0.482185,-0.594531,-0.423612
1,0.588670,0.714684,0.462147
2,0.588670,-0.135768,-0.630280
3,0.588670,0.502071,0.148630
4,-0.482185,0.291136,-0.405067
...,...,...,...
886,-0.482185,-0.253890,-0.290320
887,0.588670,-0.631865,-0.249662
888,-1.693335,NaN,-0.362597
889,2.070299,-0.322018,0.103762


#### Grouping by multiple categorical variables

- Split data into multiple levels of groups. 
- Group by sex (Male/Female) with subgroups in each according to passenger class.

df.groupby() takes in list of categorical columns to group on:

In [12]:
titanic_subset2 = titanic_df[['Sex', 'Pclass', 'Survived', 'Age', 'Fare']]
titanic_subset2.groupby(['Sex','Pclass'])

Calculate mean of attributes within these groups/subgroups:

In [13]:
grouped_df = titanic_subset2.groupby(['Sex','Pclass']).mean()
#grouped_df = titanic_subset2.groupby(by =['Sex','Pclass']).agg('mean')

grouped_df

Survived        Age        Fare
Sex    Pclass                                 
female 1       0.968085  34.611765  106.125798
       2       0.921053  28.722973   21.970121
       3       0.500000  21.750000   16.118810
male   1       0.368852  41.281386   67.226127
       2       0.157407  30.740707   19.741782
       3       0.135447  26.507589   12.661633

#### Basic Ideas of Data Shaping in Pandas
1. Wide vs. Long Formats


<div>
    <center><img src="Images/hw_wide.png" align = "center" width="400"/></center>
    <center>Wide format</center>
</div>
    

<div align>
        <center><img src="Images/hw_long.png" align = "center" width="300"/></center>
    <center>Long format</center>
</div>

#### Pivoting

- Convert from a long to a wide format:

   - DataFrame.pivot(index, columns, values):
  
 One attribute becomes index, values in other attribute becomes labels for new columns.
 
 Best to see an example:

In [16]:
value_list = [182, 160, 130, 78, 67, 52]
physical_data = pd.DataFrame(np.array([['John', 'Christopher', 'Melinda']*2, ['Height', 'Weight']*3, value_list]).T,
             columns = ['name', 'attribute', 'value'])

physical_data.head()

,name,attribute,value
0,John,Height,182
1,Christopher,Weight,160
2,Melinda,Height,130
3,John,Weight,78
4,Christopher,Height,67


This is long form. Use pivot to convert to wide format.

In [17]:
wide_form = physical_data.pivot(index = 'name', columns = 'attribute', values = 'value')
wide_form

attribute,Height,Weight
name,,
Christopher,67,160
John,182,78
Melinda,130,52


#### Melting: the inverse of pivoting.

- Take data from wide to long format.
- pd.melt(dataframe, id_vars, value_vars, var_name, value_name)

In [18]:
wide_form.reset_index(inplace = True)
wide_form

attribute,name,Height,Weight
0,Christopher,67,160
1,John,182,78
2,Melinda,130,52


In [19]:
pd.melt(wide_form, 
        id_vars = ['name'], 
        value_vars = ['Height', 'Weight'])

,name,attribute,value
0,Christopher,Height,67
1,John,Height,182
2,Melinda,Height,130
3,Christopher,Weight,160
4,John,Weight,78
5,Melinda,Weight,52


#### Pivot Tables

- When the columns you want to pivot on have non-unique entries.
- E.g., temperature as function of position X,Y for a given month but multiple measurements at each X,Y
- Want average of these measurements at each X,Y in pivoted form:

    - df.pivot_table(..., aggfunc = __)

Forest fire dataset:

Looks at temperature logged at various X, Y positions in a forest over several months.

In [20]:
forest_df = pd.read_csv('Data/forestfires.csv', usecols = ['X', 'Y', 'month', 'day', 'temp'])
inamonth_df = forest_df[(forest_df['month'] == 'mar')]

inamonth_df.head(10)

,X,Y,month,day,temp
0,7,5,mar,fri,8.2
3,8,6,mar,fri,8.3
4,8,6,mar,sun,11.4
16,5,5,mar,sat,15.1
18,6,4,mar,wed,15.9
39,4,4,mar,tue,14.1
48,4,4,mar,mon,11.8
49,4,4,mar,mon,11.0
60,2,2,mar,sun,11.5
61,2,2,mar,sun,5.5


Average temperature at (X, Y) positions for March. Organized as pivot table:

In [21]:
inamonth_df.pivot_table(index = 'X', columns = 'Y', values = 'temp', aggfunc = 'mean')

Y,2,3,4,5,6
X,,,,,
1,NaN,8.3,NaN,NaN,NaN
2,8.5,NaN,15.2000,NaN,NaN
3,NaN,NaN,13.6875,13.133333,NaN
4,NaN,11.0,14.1000,15.975000,12.30
5,NaN,NaN,14.4000,13.350000,15.10
6,NaN,15.2,14.6000,12.260000,NaN
7,NaN,11.0,13.8000,8.200000,NaN
8,NaN,NaN,NaN,NaN,12.15


#### Multiindexing
- Setting multiple columns as index
- Setting hierarchies.
- Accessing data in multi-indexed DataFrames.

Airfoil noise dataset:
- Various factors affecting sound amplitude off of airplane wings.

In [43]:
colnames = ['Frequency [Hz]', 'Angle of attack [deg]', \
            'Chord length [m]', 'Free-stream velocity [m/s]', \
            'Suction side thickness [m]', 'Sound volume [dB]']
airfoil_df = pd.read_csv('Data/airfoil_self_noise.dat', delimiter='\t', header = None, names = colnames  )

airfoil_df.head()

,Frequency [Hz],Angle of attack [deg],Chord length [m],Free-stream velocity [m/s],Suction side thickness [m],Sound volume [dB]
0,800,0.0,0.3048,71.3,0.002663,126.201
1,1000,0.0,0.3048,71.3,0.002663,125.201
2,1250,0.0,0.3048,71.3,0.002663,125.951
3,1600,0.0,0.3048,71.3,0.002663,127.591
4,2000,0.0,0.3048,71.3,0.002663,127.461


Setting multiple attributes as indices can give us flexibility in addressing the data.
- How does sound amplitude depend just on frequency, stream velocity, and foil chord length?
- Create hierarchical Multiindex:

In [23]:
col_subset = ['Frequency [Hz]', 'Free-stream velocity [m/s]', 'Angle of attack [deg]', 'Sound volume [dB]']
airfoil_df = airfoil_df[col_subset].set_index(col_subset[0:3])
airfoil_df.head()

,,,Sound volume [dB]
Frequency [Hz],Free-stream velocity [m/s],Angle of attack [deg],
800,71.3,0.0,126.201
1000,71.3,0.0,125.201
1250,71.3,0.0,125.951
1600,71.3,0.0,127.591
2000,71.3,0.0,127.461


Moved columns to index, but hierarchical structure of indices not set:
- Can be accomplished with the .sort_index() method.

In [24]:
airfoil_df = airfoil_df.sort_index()
airfoil_df.head(10)

Sound volume [dB]
Frequency [Hz] Free-stream velocity [m/s] Angle of attack [deg]                   
200            31.7                       0.0                              117.195
                                          7.3                              128.679
                                          9.5                              119.146
                                          9.9                              127.299
                                          12.3                             124.987
                                          15.4                             119.975
                                          17.4                             116.146
               39.6                       0.0                              118.129
                                          7.3                              130.989
                                          8.9                              133.420

#### Accessing via the .loc accessor on multi-indices
-DataFrame.loc[first_level_index, columns]
- Dataframe.loc[(first_level_index, second_level_index, third_level_index), columns]

In [25]:
# at frequency = 1000 Hz
airfoil_df.loc[1000, :]

Sound volume [dB]
Free-stream velocity [m/s] Angle of attack [deg]                   
31.7                       0.0                              127.365
                           0.0                              130.595
                           0.0                              130.987
                           0.0                              126.780
                           0.0                              125.127
...                                                             ...
71.3                       15.4                             131.865
                           15.6                             124.438
                           17.4                             125.816
                           19.7                             121.885
                           22.2                             134.247

[99 rows x 1 columns]

In [26]:
# sound vol vs angle of attack
# fixed at 1000 Hz, 55.5 m/s stream velocity
airfoil_df.loc[(1000, 55.5)]

,Sound volume [dB]
Angle of attack [deg],
0.0,126.966
0.0,128.290
0.0,128.583
0.0,125.586
0.0,129.800
2.0,129.002
3.0,129.329
3.3,131.129
4.0,132.134


Swapping level hierarchy:
- Look at measurement/response keeping one variable fixed and varying another.
- Swapping level hierarchy switches which we keep fixed and which we vary.


In [27]:
swapped_df = airfoil_df.swaplevel('Free-stream velocity [m/s]', 'Angle of attack [deg]').sort_index()

In [28]:
swapped_df.head()

Sound volume [dB]
Frequency [Hz] Angle of attack [deg] Free-stream velocity [m/s]                   
200            0.0                   31.7                                  117.195
                                     39.6                                  118.129
               7.3                   31.7                                  128.679
                                     39.6                                  130.989
                                     55.5                                  135.234

In [29]:
swapped_df.loc[(1000, 7.3)]

,Sound volume [dB]
Free-stream velocity [m/s],
31.7,119.099
39.6,122.539
55.5,125.194
71.3,126.838


Multi-indexing opens up many possibilities for data manipulation.

Strongly encourage you to look at supplementary material and pandas documentation!

# Methods for Combining DataFrames: `.join()`, `.merge()`, `.concat()`

Many ways to combine dataframes! Luckily, pandas has great docs: https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

# Concat

In [30]:
ds_chars = pd.read_csv('data/ds_chars.csv', index_col=0)
ds_chars

,name,HP,home_state
0,greg,200,WA
1,miles,200,WA
2,alan,170,TX
3,alison,300,DC
4,rachel,200,TX


In [31]:
prefs = pd.read_csv('data/preferences.csv', index_col=0)
prefs

,cuisine,genre
0,Greek,horror
1,Indian,scifi
2,American,fantasy
3,Thai,tech
4,Indian,documentary


Would you concat on axis = 0 or axis = 1

In [32]:
ds_full = pd.concat([ds_chars, prefs], axis=1)
ds_full

,name,HP,home_state,cuisine,genre
0,greg,200,WA,Greek,horror
1,miles,200,WA,Indian,scifi
2,alan,170,TX,American,fantasy
3,alison,300,DC,Thai,tech
4,rachel,200,TX,Indian,documentary


# join & merge

Datasets do not have to have same rows or columns.
- Just a common key (or set of keys) used to match records.

pd.merge() is the most flexible workhorse function for this:

This parameter in both `.join()` and `.merge()` tells the compiler what sort of join to effect. We'll cover this in detail when we discuss SQL.

![image showcasing how the how parameter in a join/merge would combine the two datasets, using venn-style diagrams](https://www.datasciencemadesimple.com/wp-content/uploads/2017/09/join-or-merge-in-python-pandas-1.png)
[[Image Source]](https://www.datasciencemadesimple.com/join-merge-data-frames-pandas-python/)

In [33]:
# create two datasets
import pandas as pd
df1 = pd.DataFrame({'employee': ['Chadwick', 'Bartholemew', 'Jake', 'Brunnhilde', 'Sue', 'Jimbo Jr.'],
                    'group': ['Building' ,'Accounting', 'Engineering', 'Engineering', 'HR', 'Compliance']})

df2 = pd.DataFrame({'group': ['Accounting', 'Engineering', 'HR', 'Endowment'],
                    'supervisor': ['Carly', 'Guido', 'Steve', 'Eileen']})
df3 = pd.DataFrame({'name': ['Brunnhilde', 'Bartholemew', 'Jake', 'Sue'],
                    'hire_date': [2004, 2008, 2012, 2014]})


In [34]:
df1

,employee,group
0,Chadwick,Building
1,Bartholemew,Accounting
2,Jake,Engineering
3,Brunnhilde,Engineering
4,Sue,HR
5,Jimbo Jr.,Compliance


In [35]:
df2

,group,supervisor
0,Accounting,Carly
1,Engineering,Guido
2,HR,Steve
3,Endowment,Eileen


In [36]:
pd.merge(df1, df2, how = 'inner', on = 'group')

,employee,group,supervisor
0,Bartholemew,Accounting,Carly
1,Jake,Engineering,Guido
2,Brunnhilde,Engineering,Guido
3,Sue,HR,Steve


In [37]:
pd.merge(df1, df2, how = 'left', on = 'group')

,employee,group,supervisor
0,Chadwick,Building,NaN
1,Bartholemew,Accounting,Carly
2,Jake,Engineering,Guido
3,Brunnhilde,Engineering,Guido
4,Sue,HR,Steve
5,Jimbo Jr.,Compliance,NaN


In [38]:
pd.merge(df1, df2, how = 'right', on = 'group')

,employee,group,supervisor
0,Bartholemew,Accounting,Carly
1,Jake,Engineering,Guido
2,Brunnhilde,Engineering,Guido
3,Sue,HR,Steve
4,NaN,Endowment,Eileen


merge on key with different label:

In [39]:
df1


,employee,group
0,Chadwick,Building
1,Bartholemew,Accounting
2,Jake,Engineering
3,Brunnhilde,Engineering
4,Sue,HR
5,Jimbo Jr.,Compliance


In [40]:
df3

,name,hire_date
0,Brunnhilde,2004
1,Bartholemew,2008
2,Jake,2012
3,Sue,2014


In [41]:
pd.merge(df1, df3, left_on = 'employee', right_on = 'name', how = 'inner')
# what names will ne included?

,employee,group,name,hire_date
0,Bartholemew,Accounting,Bartholemew,2008
1,Jake,Engineering,Jake,2012
2,Brunnhilde,Engineering,Brunnhilde,2004
3,Sue,HR,Sue,2014


Can do a bit more with merge: 
- merge matching on multiple columns as opposed to one.
- df1.join(df2, how = ' '): similar to merge but less flexible. Joins on index. Faster than merge.


In [42]:
df1.set_index('group').join(df2.set_index('group'), how = 'inner')

,employee,supervisor
group,,
Accounting,Bartholemew,Carly
Engineering,Jake,Guido
Engineering,Brunnhilde,Guido
HR,Sue,Steve


Data in real life can be messy:

- Often keys have mispellings or don't exactly match up
- Determine whether key is similar enough.
- Then link record if true.